In [1]:
from src import init_data
import pyterrier as pt
from src.pyterrier.pyterrier_math_formula_coding import translate_latex
from arqmath_code.post_reader_record import DataReaderRecord

In [2]:
topic_reader, data_reader = init_data(task=1)

reading users
reading comments
reading votes
reading post links
reading posts


In [3]:
if not pt.started():
  pt.init()

         Using cpu = 'i386' instead!


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [ ]:
def get_arqmath_answers_as_iterable(data_reader: DataReaderRecord):
    for answer in data_reader.get_all_answer_posts():
        if answer.body is not None and answer.post_id is not None and answer.parent_id is not None and answer.votes is not None:
            indexed_body = translate_latex(answer.body)

            yield {'docno': answer.post_id,
                   'text': indexed_body,
                   'origtext': answer.body,
                   'parentno': answer.parent_id,
                   'votes': answer.votes
                   }

In [4]:
import os
from src.pyterrier.config import ROOT_DIRECTORY

index_path = f"{ROOT_DIRECTORY}/index/arqmath_indexV1"
if not os.path.exists(index_path + "/data.properties"):
    indexer = pt.IterDictIndexer(index_path, overwrite=True)
    dict_field_names = ["docno", "text", "origtext", "parentno", "votes"]
    index_ref_arqmath = indexer.index(get_arqmath_answers_as_iterable(data_reader))

else:
    index_ref_arqmath = pt.IndexRef.of(index_path + "/data.properties")

index_ref_arqmath.toString()

'/Users/I518152/Documents/GitHub/information-retrieval/index/arqmath_indexV1/data.properties'

In [5]:
index = pt.IndexFactory.of(index_ref_arqmath)
type(index)

jnius.reflect.org.terrier.structures.Index

In [6]:
print(index.getCollectionStatistics().toString())

Number of documents: 1191594
Number of terms: 273825
Number of postings: 65910187
Number of fields: 1
Number of tokens: 546619634
Field names: [text]
Positions:   false



In [ ]:
for kv in index.getLexicon():
  print(f"{kv.getKey()} -> { kv.getValue().toString()}")

In [8]:
index.getLexicon()["number"].toString()

'term325 Nt=170604 TF=380771 maxTF=2147483647 @{0 128783516 0} TFf=380771'

In [9]:
br = pt.BatchRetrieve(index, wmodel="BM25")
br.search("linear algebra")

,qid,docid,docno,rank,score,query
0,1,710849,1701210,0,18.071328,linear algebra
1,1,136794,256695,1,18.038732,linear algebra
2,1,142024,266868,2,17.859869,linear algebra
3,1,103414,265736,3,17.835491,linear algebra
4,1,90558,161735,4,17.801233,linear algebra
...,...,...,...,...,...,...
995,1,230186,453086,995,14.383920,linear algebra
996,1,72175,126497,996,14.379584,linear algebra
997,1,24928,467209,997,14.376588,linear algebra
998,1,591735,1373861,998,14.375329,linear algebra


In [10]:
data_reader.post_parser.map_just_answers[1701210].body

'<p>Well, let\'s try to imagine how to <em>avoid</em> linear algebra. </p>  <p>Anything that involves functions of <span class="math-container" id="15678330">2</span> or more variables, that can be differentiated, will very soon use linear operators or matrices.  No geometry!</p>  <p>Also, no algebra or number theory or topology.  Almost anything in algebra or its client subjects that involves associative (not necessarily commutative) multiplication, such as composition of functions, requires the simpler case of linear algebra as a preliminary. </p>  <p>That leaves calculus of one real or complex variable. There, it will still be hard to avoid linear algebra, unless you also avoid recursion relations, differential and difference equations, operators on the functions, measures, Fourier and Laplace transforms, special function and orthogonal polynomials, discrete or finite-element methods, calculus of variations.  Sooner or later one meets linear-algebra concepts like eigenvalue, linear 